<H1><center><font color='limegreen'>Bank Marketing Campaign Analysis</font>
<br>----------

<H3><font color='limegreen'>By : Group Delta</font>
<br>Fathur Rizky Maulana
<br>Alfian Abdul Ghaffar
<br>Dafiq Alfaiz
<br>-------------------------

## **Contents**

0. Initial Setup
1. Business Problem Understanding
2. Data Understanding
3. Data Preprocessing
4. Modeling
5. Conclusion
6. Recommendation

----
----

### **`Initial Setup`**

In [2]:
# Import library eksplorasi dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt #Membuat grafik
import seaborn as sns #Membuat grafik
import missingno as msno #Cek missing value

### **`Business Problem Understanding`**

#### **Context**

#### **Problem Statement**

#### **Project Stakeholders:**

#### **Goals**

#### **Analytic Approach**

#### **Metric Evaluation**

### **`Data Understanding`**

Dataset dan deskripsi dapat di akses pada link berikut:
<br>
[Bank Marketing Campaign](https://www.kaggle.com/datasets/volodymyrgavrysh/bank-marketing-campaigns-dataset)

| Columns                                            | Definition                                                   | 
| ------------------------------------------------- | ------------------------------------------------------------ |
| `age` | Umur nasabah (`tahun`) |
| `job` | jenis pekerjaan nasabah |
| `marital` | status pernikahan nasabah |
| `education` | level pendidikan nasabah |
| `default` | Apakah memiliki riwayat kredit macet |
| `housing` | Apakah memiliki pinjaman dana untuk rumah |
| `loan` | Apakah memiliki pinjaman dana untuk pribadi |
| `contact` | Tipe device untuk berkomunikasi |
| `month` | Bulan terakhir kali dihubungi |
| `day_of_week` | Minggu terakhir kali dihubungi |
| `duration` | Lama waktu ketika dihubungi (`detik`) |
| `campaign` | Jumlah kontak yang dilakukan selama campaign |
| `pdays` | Jumlah hari setelah terakhir kali dihubungi |
| `previous` | Jumlah kontak yang dilakukan sebelum campaign |
| `poutcome` | Hasil campaign sebelumnya |
| `emp.var.rate` | Tingkat variasi lapangan kerja |
| `cons.price.idx` | Indeks harga konsumen |
| `cons.conf.idx` | Indeks kepercayaan konsumen |
| `euribor3m` | Tarif euribor 3 bulan |
| `nr.employed` | Jumlah karyawan |
| `y` | Status berlangganan deposito |

**`y`** == `Yes` : Nasabah berlangganan deposito berjangka, `No` : Nasabah tidak berlangganan deposito berjangka
<br>

In [4]:
# Import file CSV

df = pd.read_csv("D:\Purwadhika JCDS\Capstone Project\Final Project\data\\raw\\bank-additional-full.csv", sep=";")
print(f'Jumlah baris dan kolom: {df.shape}')

df.sample(10)

Jumlah baris dan kolom: (41188, 21)


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
15809,47,housemaid,divorced,basic.4y,unknown,no,no,cellular,jul,mon,...,1,999,0,nonexistent,1.4,93.918,-42.7,4.960,5228.1,no
15290,49,admin.,divorced,high.school,no,no,no,telephone,jul,thu,...,1,999,0,nonexistent,1.4,93.918,-42.7,4.958,5228.1,no
1156,58,admin.,married,university.degree,no,no,yes,telephone,may,thu,...,2,999,0,nonexistent,1.1,93.994,-36.4,4.855,5191.0,no
40966,74,retired,married,university.degree,no,yes,no,cellular,oct,fri,...,2,3,2,success,-1.1,94.601,-49.5,0.993,4963.6,no
37935,71,retired,married,basic.4y,no,yes,no,telephone,sep,thu,...,1,999,0,nonexistent,-3.4,92.379,-29.8,0.809,5017.5,no
7850,30,services,married,high.school,no,no,no,telephone,jun,mon,...,2,999,0,nonexistent,1.4,94.465,-41.8,4.865,5228.1,no
17815,54,housemaid,married,basic.4y,unknown,no,no,cellular,jul,tue,...,2,999,0,nonexistent,1.4,93.918,-42.7,4.961,5228.1,no
12696,48,admin.,divorced,high.school,no,no,no,telephone,jul,mon,...,2,999,0,nonexistent,1.4,93.918,-42.7,4.960,5228.1,no
21960,29,technician,married,university.degree,no,yes,yes,cellular,aug,wed,...,2,999,0,nonexistent,1.4,93.444,-36.1,4.964,5228.1,no
40982,74,retired,divorced,basic.4y,no,yes,no,cellular,oct,mon,...,2,999,0,nonexistent,-1.1,94.601,-49.5,1.000,4963.6,no


#### **Initial Assessment**

In [6]:
# Check karakteristik dataset

pd.DataFrame(
        {
        'feature': df.columns.values,
        'data_type': df.dtypes.values,
        'total_row': df.isna().count().values,
        'total_null': df.isna().sum().values,
        '%null_value': df.isna().mean().values * 100,
        'n_unique': df.nunique().values,
        'sample_unique': [df[col].unique() for col in df.columns]
        }
    )

,feature,data_type,total_row,total_null,%null_value,n_unique,sample_unique
0,age,int64,41188,0,0.0,78,"[56, 57, 37, 40, 45, 59, 41, 24, 25, 29, 35, 5..."
1,job,object,41188,0,0.0,12,"[housemaid, services, admin., blue-collar, tec..."
2,marital,object,41188,0,0.0,4,"[married, single, divorced, unknown]"
3,education,object,41188,0,0.0,8,"[basic.4y, high.school, basic.6y, basic.9y, pr..."
4,default,object,41188,0,0.0,3,"[no, unknown, yes]"
5,housing,object,41188,0,0.0,3,"[no, yes, unknown]"
6,loan,object,41188,0,0.0,3,"[no, yes, unknown]"
7,contact,object,41188,0,0.0,2,"[telephone, cellular]"
8,month,object,41188,0,0.0,10,"[may, jun, jul, aug, oct, nov, dec, mar, apr, ..."
9,day_of_week,object,41188,0,0.0,5,"[mon, tue, wed, thu, fri]"


__Insight :__
<br>

* Numerical Variable:
<br>
    - `Discrete` : age, campaign, pdays, previous
<br>
    - `Continue` : duration, emp.var.rate, cons.price.idx, cons.conf.idx, euribor3m, nr.employed
* Categorical variable:
<br>
    - `Nominal` : job, marital, default, housing, loan, contact, month, day_of_week, poutcome, y
<br>
    - `Ordinal` : education
* `Target`: y
* Terdapat total 41.188 jumlah baris pada dataset
* Tipe data untuk setiap variabel sudah sesuai
* Tidak terdapat missing value pada setiap kolom (*analisis lebih lanjut*)
* Pada kolom "*duration*" memiliki unique value yang sangat banyak sebesar 1.544 unique value. Maka kita akan kelompokan berdasarkan kelasnya
* Beberapa kolom yang akan dikelompokan berdasarkan kelasnya antara lain: 

Lalu kita akan coba melihat statistik deskriptif sederhana dari dataset ini

In [7]:
# Deskriptif statistik

display(
    # numerical variable
    df.describe(),
    # categorical variable
    df.describe(include='object')
)

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
count,41188.00000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000
mean,40.02406,258.285010,2.567593,962.475454,0.172963,0.081886,93.575664,-40.502600,3.621291,5167.035911
std,10.42125,259.279249,2.770014,186.910907,0.494901,1.570960,0.578840,4.628198,1.734447,72.251528
min,17.00000,0.000000,1.000000,0.000000,0.000000,-3.400000,92.201000,-50.800000,0.634000,4963.600000
25%,32.00000,102.000000,1.000000,999.000000,0.000000,-1.800000,93.075000,-42.700000,1.344000,5099.100000
50%,38.00000,180.000000,2.000000,999.000000,0.000000,1.100000,93.749000,-41.800000,4.857000,5191.000000
75%,47.00000,319.000000,3.000000,999.000000,0.000000,1.400000,93.994000,-36.400000,4.961000,5228.100000
max,98.00000,4918.000000,56.000000,999.000000,7.000000,1.400000,94.767000,-26.900000,5.045000,5228.100000


,job,marital,education,default,housing,loan,contact,month,day_of_week,poutcome,y
count,41188,41188,41188,41188,41188,41188,41188,41188,41188,41188,41188
unique,12,4,8,3,3,3,2,10,5,3,2
top,admin.,married,university.degree,no,yes,no,cellular,may,thu,nonexistent,no
freq,10422,24928,12168,32588,21576,33950,26144,13769,8623,35563,36548
